## ETL Development

This file serves as as draft for designing & development of project ETLs - transformations & preprocessing of data for modeling. The ETL should be part of runnable  
pipeline for model encapsulation & enviroment isolation purposes (leak of statistics).

Includes:
- Transformation ETLs
- Encoding ETLs

ETLs must be part of pipeline to ensure statistical calculation only on train dataset.

In [2]:
import os
import sys

sys.dont_write_bytecode =True

import numpy as np
import pandas as pd

loading data

In [3]:
DATA_DIR = './Data/'
FILE = 'spotify_tracks_kaggle_weekly.csv'

In [4]:
data = pd.read_csv(DATA_DIR + FILE)

columns to be dropped:

- track_id : ID
- artwork_url : url data - not intended for modelling purposes
- track_url : url data - not intended for modelling purposes
- track_name : user-friendly ID - not intended for modelling purposes (could be used in NLP analysis, however that will not be subject of this project)

In [12]:
DROP_COLUMNS = ['track_id', 'artwork_url', 'track_url', 'track_name']

In [13]:
data = data.drop(DROP_COLUMNS, axis=1, errors='ignore')

In [15]:
data.head()

,artist_name,year,popularity,album_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,language
0,Anirudh Ravichander,2024,59,"Leo Das Entry (From ""Leo"")",0.0241,0.753,97297.0,0.970,0.055300,8.0,0.1000,-5.994,0.0,0.1030,110.997,4.0,0.459,Tamil
1,"Anirudh Ravichander, Pravin Mani, Vaishali Sri...",2024,47,AAO KILLELLE,0.0851,0.780,207369.0,0.793,0.000000,10.0,0.0951,-5.674,0.0,0.0952,164.995,3.0,0.821,Tamil
2,"Anirudh Ravichander, Anivee, Alvin Bruno",2024,35,Mayakiriye Sirikiriye (Orchestral EDM),0.0311,0.457,82551.0,0.491,0.000000,2.0,0.0831,-8.937,0.0,0.1530,169.996,4.0,0.598,Tamil
3,"Anirudh Ravichander, Bharath Sankar, Kabilan, ...",2024,24,Scene Ah Scene Ah (Experimental EDM Mix),0.2270,0.718,115831.0,0.630,0.000727,7.0,0.1240,-11.104,1.0,0.4450,169.996,4.0,0.362,Tamil
4,"Anirudh Ravichander, Benny Dayal, Leon James, ...",2024,22,Gundellonaa X I Am a Disco Dancer (Mashup),0.0153,0.689,129621.0,0.748,0.000001,7.0,0.3450,-9.637,1.0,0.1580,128.961,4.0,0.593,Tamil


### Encoding

Due to the nature of the dataset multiple categorical (numerical as well) columns will require specific encoding method to convey information correctly.

- Categorical data:
    - artist_name
    - album_name
    - language


- Numerical data:
    - key
    - mode